In [1]:
import pandas as pd
import random

pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_columns', 50)

In [2]:
# rolls
roll = {
    '📏': 'range',
    '❤': 'damage',
    '⚡': 'surge',
    '❌': 'miss',
    '🗡': 'pierce',
}

red = ['❌', '❤❤❤❤', '❤❤📏📏', '❤📏📏⚡', '❤❤❤📏', '❤❤❤📏⚡']
blue = ['❌', '📏📏📏📏⚡', '❤📏📏📏⚡', '❤📏📏📏⚡', '❤❤📏📏', '❤❤📏']
white = ['❌', '❤📏📏📏⚡', '❤📏📏📏⚡', '❤❤❤📏⚡', '❤❤❤📏⚡', '❤❤📏📏']
green = ['❤❤❤', '❤❤⚡', '❤❤❤', '❤❤⚡', '❤❤📏', '❤📏']
yellow = ['📏📏⚡', '📏⚡', '📏📏⚡', '📏📏❤', '📏📏📏', '📏📏📏']
black = ['❤📏', '❤📏', '❤📏', '', '⚡', '⚡']
clear = ['❌', '❌', '', '', '', '']

In [3]:
# dice classes

class Die:
    def __init__(self, colour, possible_results):
        self.colour = colour
        self.POSSIBLE_RESULTS = possible_results
    
    def roll(self):
        return random.choice(self.POSSIBLE_RESULTS)


class Dice(Die):
    def __init__(self, n_red=0, n_blue=0, n_white=0, n_green=0, n_yellow=0, n_black=0, n_clear=0):
        self._dice = list()
        self.get_dice(n_red, n_blue, n_white, n_green, n_yellow, n_black, n_clear)
        self.current_roll = None
    
    def get_dice(self, n_red, n_blue, n_white, n_green, n_yellow, n_black, n_clear):
        for colour in ['red', 'blue', 'white', 'green', 'yellow', 'black', 'clear']:
            self._dice = self._dice + [Die(colour, eval(colour)) for _ in range(eval('n_'+colour))]

    def roll(self):
        result = []
        for die in self._dice:
            die_result = die.roll()
            result.append(die_result)
        self.current_roll = result
        return self.current_roll


In [4]:
# monster class

class Monster:
    """Monsters don't use effects (surges just lead to threat, which isn't tracked in this AI variant)"""
    def __init__(self, dice_obj, modifiers=None, type=None):
        self.dice = dice_obj
        self.modifiers = modifiers
        self.current_roll = None
        self.current_totals = None
        self.type = type
        self.roll()
        
    def roll(self):
        if self.dice:
            result = self.dice.roll()
        else:
            result = []
        if self.modifiers:
            modifier_list = []
            for k,v in self.modifiers.items():
                for _ in range(v):
                    modifier_list.append(k)
            result.extend(modifier_list)
        self.current_roll = result    
        self.calc_totals()
        return self.current_roll
    
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()
            
    def calc_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    final_results[result] = final_results.get(result, 0) + 1
        self.current_totals = final_results

    def print_totals(self):
        if self.current_totals == None:
            self.calc_totals()
        for result in self.current_totals.items():
            print(result)
        print()

    def results(self):        
        results = []
        for r in self.current_roll:
            results += [x for x in r]
        results = ''.join(sorted(results, reverse=True))
        
        if '🗡' in results:
            columns = ['❌', '❤', '🗡', '📏', '⚡']
            output = pd.DataFrame([[results.count('❌'), results.count('❤'), results.count('🗡'), results.count('📏'), results.count('⚡')]], columns=columns, dtype='int64')
        else:
            columns = ['❌', '❤', '📏', '⚡']
            output = pd.DataFrame([[results.count('❌'), results.count('❤'), results.count('📏'), results.count('⚡')]], columns=columns, dtype='int64')

        if self.type == 'melee':
            output = output.drop('📏', axis=1)

        return output


In [6]:
# simulation functions

def simulation(obj, n_trials=100):
    obj.roll()
    results = pd.DataFrame(obj.results().max()).T
    for _ in range(n_trials-1):
        obj.roll()
        results = pd.concat([results, pd.DataFrame(obj.results().max()).T])
    return results

def summarise_simulation(dataframe, percentiles=[0.01]+[x/20 for x in range(1, 20)] + [0.99]):
    cols = ['mean', 'std']+[str(int(x*100))+'%' for x in percentiles]+['max']
    output = dataframe.describe(percentiles=percentiles).T
    return pd.concat([output.iloc[:, 1:3], output.iloc[:, 3:].astype(int)], axis=1)[cols]

def print_simulation_results(kwargs_dict, n_trials=1000):
    for monster, kwargs in kwargs_dict.items():
        print()
        print(monster)
        display(summarise_simulation(simulation(Monster(**kwargs), n_trials=n_trials)))

def calculate_range(kwargs_dict, n_trials=100, n_percentile=50):
    percentile = str(n_percentile)+'%'
    output_dict = dict()
    for monster, kwargs in kwargs_dict.items():
        if kwargs.get('type', False) and kwargs['type'] == 'melee':
            output_dict[monster] = 0
        else:
            df = summarise_simulation(simulation(Monster(**kwargs), n_trials=n_trials))
            output_dict[monster] = df.loc[['📏'], [percentile]].iloc[0].iloc[0]

    return output_dict

In [30]:
# generate kwargs for simulation
generate_kwargs = False

if generate_kwargs:
    MONSTERS = pd.read_csv('monster_import.csv')
    MONSTERS.name = MONSTERS.name.str.replace(' ', '_').str.lower()

    print('monsters = {')
    for i, row in MONSTERS.iloc[:, [0, 12, 13, 14, 15, 16, 17, 18, 19, 20]].iterrows():
        print("    '" + row['name'] + "': {")
        
        # dice kwargs
        kwargs = ', '.join([f"{x}={str(int(y))}" for x, y in row[1:7].items() if pd.notna(y)])
        print(f"        'dice_obj': Dice({kwargs}),")

        # modifiers
        modifiers = row['abilities']
        print(f"        'modifiers': {{{modifiers}}},")

        # type
        print(f"        'type': '{row['attack_type']}'")

        print("    },")
    print('}')
    

In [8]:
# monster kwargs

monsters = {
    'red_die': {
        'dice_obj': Dice(n_red=1),
    },
    'blue_die': {
        'dice_obj': Dice(n_blue=1),
    },
    'white_die': {
        'dice_obj': Dice(n_white=1),
    },
    'green_die': {
        'dice_obj': Dice(n_green=1),
    },
    'yellow_die': {
        'dice_obj': Dice(n_yellow=1),
    },
    'black_die': {
        'dice_obj': Dice(n_black=1),
    },
    'clear_die': {
        'dice_obj': Dice(n_clear=1),
    },

    'bane_spider': {
        'dice_obj': Dice(n_white=1, n_green=1),
        'modifiers': {},
        'type': 'magic'
    },
    'bane_spider_master': {
        'dice_obj': Dice(n_white=1, n_green=1, n_black=1),
        'modifiers': {},
        'type': 'magic'
    },
    'beastman': {
        'dice_obj': Dice(n_red=1, n_green=1),
        'modifiers': {'❤': 1},
        'type': 'melee'
    },
    'beastman_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=1),
        'modifiers': {'❤': 2},
        'type': 'melee'
    },
    'blood_ape': {
        'dice_obj': Dice(n_red=1, n_green=1, n_yellow=1),
        'modifiers': {},
        'type': 'melee'
    },
    'blood_ape_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_yellow=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'chaos_beast_melee': {  # TODO: maybe figure out morph or just ignore forever and write specific AI for this dude
        'dice_obj': Dice(n_red=1, n_green=2),
        'modifiers': {'❤': 1, '📏': 1},
        'type': 'unknown'
    },
    'chaos_beast_master_melee': {
        'dice_obj': Dice(n_red=1, n_green=2, n_black=1),
        'modifiers': {'❤': 2, '📏': 2},
        'type': 'unknown'
    },
    'chaos_beast_range': {
        'dice_obj': Dice(n_blue=1, n_yellow=2),
        'modifiers': {'❤': 1, '📏': 1},
        'type': 'unknown'
    },
    'chaos_beast_master_range': {
        'dice_obj': Dice(n_blue=1, n_yellow=2, n_black=1),
        'modifiers': {'❤': 2, '📏': 2},
        'type': 'unknown'
    },
    'dark_priest': {
        'dice_obj': Dice(n_white=1, n_black=3),
        'modifiers': {},
        'type': 'magic'
    },
    'dark_priest_master': {
        'dice_obj': Dice(n_white=1, n_black=5),
        'modifiers': {},
        'type': 'magic'
    },
    'deep_elf': {
        'dice_obj': Dice(n_red=1, n_green=1),
        'modifiers': {'🗡': 3},
        'type': 'melee'
    },
    'deep_elf_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=1),
        'modifiers': {'🗡': 6},
        'type': 'melee'
    },
    'demon': {
        'dice_obj': Dice(n_white=1, n_green=1, n_yellow=2),
        'modifiers': {'❤': 2, '📏': 2},
        'type': 'magic'
    },
    'demon_master': {
        'dice_obj': Dice(n_white=1, n_green=2, n_yellow=1, n_black=1),
         'modifiers': {'❤': 2, '📏': 2},
        'type': 'magic'
    },
    'dragon': {
        'dice_obj': Dice(n_blue=1, n_green=1, n_yellow=2),
        'modifiers': {'🗡': 5},
        'type': 'ranged'
    },
    'dragon_master': {
        'dice_obj': Dice(n_blue=1, n_green=2, n_yellow=1, n_black=1),
        'modifiers': {'🗡': 5},
        'type': 'ranged'
    },
    'ferrox': {
        'dice_obj': Dice(n_red=1, n_green=1),
        'modifiers': {},
        'type': 'melee'
    },
    'ferrox_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'giant': {
        'dice_obj': Dice(n_red=1, n_green=2, n_yellow=1),
        'modifiers': {},
        'type': 'melee'
    },
    'giant_master': {
        'dice_obj': Dice(n_red=1, n_green=2, n_yellow=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'golem': {
        'dice_obj': Dice(n_red=1, n_green=1, n_yellow=1),
        'modifiers': {},
        'type': 'melee'
    },
    'golem_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_yellow=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'hell_hound': {
        'dice_obj': Dice(n_blue=1, n_yellow=1),
        'modifiers': {'🗡': 3},
        'type': 'magic'
    },
    'hell_hound_master': {
        'dice_obj': Dice(n_blue=1, n_yellow=1, n_black=1),
        'modifiers': {'🗡': 4},
        'type': 'magic'
    },
    'ice_wyrm': {
        'dice_obj': Dice(n_blue=1, n_green=2, n_black=1),
        'modifiers': {},
        'type': 'ranged'
    },
    'ice_wyrm_master': {
        'dice_obj': Dice(n_blue=1, n_green=2, n_black=2),
        'modifiers': {},
        'type': 'ranged'
    },
    'kobold': {
        'dice_obj': Dice(n_red=1),
        'modifiers': {},
        'type': 'melee'
    },
    'kobold_master': {
        'dice_obj': Dice(n_red=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'lava_beetle': {
        'dice_obj': Dice(n_blue=1, n_green=1),
        'modifiers': {},
        'type': 'ranged'
    },
    'lava_beetle_master': {
        'dice_obj': Dice(n_blue=1, n_green=1, n_black=1),
        'modifiers': {},
        'type': 'ranged'
    },
    'manticore': {
        'dice_obj': Dice(n_blue=1, n_green=1, n_yellow=1),
        'modifiers': {'🗡': 2},
        'type': 'ranged'
    },
    'manticore_master': {
        'dice_obj': Dice(n_blue=1, n_green=1, n_yellow=1, n_black=1),
        'modifiers': {'🗡': 3},
        'type': 'ranged'
    },
    'medusa': {
        'dice_obj': Dice(n_white=1, n_green=1, n_yellow=1),
        'modifiers': {},
        'type': 'magic'
    },
    'medusa_master': {
        'dice_obj': Dice(n_white=1, n_green=1, n_yellow=1, n_black=1),
        'modifiers': {},
        'type': 'magic'
    },
    'naga': {
        'dice_obj': Dice(n_white=1, n_green=1, n_yellow=1),
         'modifiers': {'❤': 1, '📏': 1},
        'type': 'magic'
    },
    'naga_master': {
        'dice_obj': Dice(n_white=1, n_green=1, n_yellow=1, n_black=1),
         'modifiers': {'❤': 2, '📏': 2},
        'type': 'magic'
    },
    'ogre': {
        'dice_obj': Dice(n_red=1, n_green=2),
        'modifiers': {},
        'type': 'melee'
    },
    'ogre_master': {
        'dice_obj': Dice(n_red=1, n_green=2, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'razorwing': {
        'dice_obj': Dice(n_red=1, n_yellow=2),
        'modifiers': {},
        'type': 'melee'
    },
    'razorwing_master': {
        'dice_obj': Dice(n_red=1, n_yellow=2, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'shade': {
        'dice_obj': Dice(n_white=1, n_yellow=1),
        'modifiers': {'🗡': 2},
        'type': 'magic'
    },
    'shade_master': {
        'dice_obj': Dice(n_white=1, n_yellow=1, n_black=1),
        'modifiers': {'🗡': 4},
        'type': 'magic'
    },
    'skeleton': {
        'dice_obj': Dice(n_blue=1, n_green=1),
        'modifiers': {'🗡': 3, '📏': 1},
        'type': 'ranged'
    },
    'skeleton_master': {
        'dice_obj': Dice(n_blue=1, n_green=1, n_black=1),
        'modifiers': {'🗡': 3, '📏': 2},
        'type': 'ranged'
    },
    'sorcerer': {
        'dice_obj': Dice(n_white=1, n_yellow=1),
        'modifiers': {'❤': 3, '📏': 3},
        'type': 'magic'
    },
    'sorcerer_master': {
        'dice_obj': Dice(n_white=1, n_yellow=1, n_black=1),
        'modifiers': {'❤': 4, '📏': 4},
        'type': 'magic'
    },
    'troll': {
        'dice_obj': Dice(n_red=1, n_yellow=2),
        'modifiers': {},
        'type': 'melee'
    },
    'troll_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_yellow=1),
        'modifiers': {},
        'type': 'melee'
    },
    'wendigo': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=1),
        'modifiers': {},
        'type': 'melee'
    },
    'wendigo_master': {
        'dice_obj': Dice(n_red=1, n_green=1, n_black=3),
        'modifiers': {},
        'type': 'melee'
    }
}


In [9]:
print_simulation_results(monsters, n_trials=100)


red_die


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
❤,2.1,1.3,0,0,0,0,1,1,1,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4
📏,1.0,0.8,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2
⚡,0.3,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1



blue_die


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.1,0.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
❤,1.0,0.8,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2
📏,2.3,1.3,0,0,0,1,1,1,1,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4
⚡,0.5,0.5,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1



white_die


...

,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
❤,4.4,1.6,1,2,2,2,3,3,4,4,4,4,4,5,5,5,6,6,6,6,6,7,7,8
⚡,1.2,0.8,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3



wendigo


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.1,0.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
❤,4.7,1.5,1,2,3,3,3,4,4,4,4,4,5,5,5,6,6,6,6,6,7,7,7,8
⚡,1.1,0.9,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3



wendigo_master


,mean,std,1%,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,99%,max
❌,0.2,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
❤,5.9,2.0,2,2,3,4,4,4,5,5,6,6,6,6,7,7,7,7,8,8,8,9,10,10
⚡,1.6,1.0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,4,4


In [21]:
# calculate range for monsters
range_50 = calculate_range(monsters)
range_10 = calculate_range(monsters, n_percentile=10)

df_range = pd.merge(
    pd.DataFrame(range_10.items(), columns=['name', 'low_range']),
    pd.DataFrame(range_50.items(), columns=['name', 'high_range']),
    on='name'
)

In [29]:
# maybe 20th percentile is a bit better payoff between distance and accuracy
# TODO: dragon and hellhound just have area attack, range is 3ish regardless of roll
# TODO: incorporate range into monster data - maybe just paste this into in csv (with above-noted fixes)?
pd.DataFrame(calculate_range(monsters, n_percentile=20).items(), columns=['name', 'range'])

,name,range
0,red_die,0
1,blue_die,1
2,white_die,1
3,green_die,0
4,yellow_die,2
5,black_die,0
6,clear_die,0
7,bane_spider,1
8,bane_spider_master,1
9,beastman,0
